# Imports

In [1]:
import numpy as np
import pandas as pd
import pgeocode
import plotly.graph_objects as go

# 2017

# Dataset
### CSV file needs some trimming before using it here. Columns not used and empty columns are removed manually

In [2]:
data17 = pd.read_csv("../Data/2017/2017_trimmed.csv", header=[0,1])

# Working dataset copy

In [3]:
df = data17.copy()

In [4]:
df.head()

Name of Area Total Population                \
      Geo_NAME    SE_A00001_001 SE_A00002_001   
0  00601 ZCTA5            17599         17599   
1  00602 ZCTA5            39209         39209   
2  00603 ZCTA5            50135         50135   
3  00606 ZCTA5             6304          6304   
4  00610 ZCTA5            27590         27590   

  Population Density (Per Sq. Mile)   Area (Land)   Area Total:  \
                      SE_A00002_002 SE_A00002_003 SE_A00003_001   
0                          273.4985     64.347708      64.65632   
1                         1280.7820     30.613331      32.33005   
2                         1585.8370     31.614216      31.68504   
3                          148.9987     42.309101      42.31392   
4                          768.1858     35.915790      37.52661   

  Area Total: Area (Land) Area Total: Area (Water) Total Population:  \
            SE_A00003_002            SE_A00003_003     SE_A02001_001   
0                64.34771                 0.308610             17599   
1                30.61333                 1.716716             39209   
2                31.61422                 0.070821             50135   
3                42.30910                 0.004821              6304   
4                35.91579                 1.610819             27590   

  Total Population: Male  ... Own Children under 18 Years  \
           SE_A02001_002  ...               SE_A10065_001   
0                   8809  ...                        3548   
1                  19231  ...                        5961   
2                  24497  ...                        8973   
3                   3148  ...                        1109   
4                  13332  ...                        4524   

  Own Children under 18 Years: Children Living with Single Parents  \
                                                     SE_A10065_002   
0                                               2028                 
1                                               2756                 
2                                               4913                 
3                                                636                 
4                                               2100                 

     Households Households: 1-Person Household Households: 2-Person Household  \
  SE_A10066_001                  SE_A10066_002                  SE_A10066_003   
0          5818                           1416                           1909   
1         12719                           2938                           4219   
2         19009                           5037                           6152   
3          1959                            425                            701   
4          9120                           2208                           2855   

  Households: 3-Person Household Households: 4-Person Household  \
                   SE_A10066_004                  SE_A10066_005   
0                           1318                            847   
1                           2436                           2277   
2                           3999                           2418   
3                            337                            328   
4                           1793                           1350   

  Households: 5-Person Household Households: 6-Person Household  \
                   SE_A10066_006                  SE_A10066_007   
0                            242                             54   
1                            598                            135   
2                           1171                            140   
3                            138                              8   
4                            685                            128   

  Households: 7-or-More Person Household  
                           SE_A10066_008  
0                                     32  
1                                    116  
2                                     92  
3                                     22 

# Preprocess Data

## Extract zip codes

In [5]:
df.insert(0, 'zip_code', df[('Name of Area', 'Geo_NAME')].astype(str).str[:5])
df.drop('Name of Area', level = 0, axis =1, inplace=True)

In [6]:
df = df.loc[pd.to_numeric(df.zip_code).between(10000,20000)] # Filtering rows to limit the data we work on. Selecting group of states starting with 1

In [7]:
df.set_index('zip_code', inplace=True)

In [8]:
df.shape

(3658, 2177)

### Extracting only the tables required for the analysis from the dataset

In [9]:
#OLD LIST table_names = ["SE_F13004", "SE_A17005", "SE_A03001", "SE_A10021", "SE_A10027", "SE_A10035", "SE_A12001", "SE_A12003", "SE_A12004", "SE_A12005", "SE_A12006", "SE_A12007", "SE_A12008", "SE_A12009", "SE_A12010", "SE_A12011", "SE_A12012", "SE_A12013", "SE_A12014", "SE_A12015", "SE_A12016", "SE_A12017", "SE_A12018", "SE_A12019", "SE_A00002", "SE_A14010"]
table_names = ['SE_B13004', 'SE_B18002', 'SE_A14009', 'SE_A03001', 'SE_A13002', 'SE_A13007', 'SE_A14006', 'SE_A14028', 'SE_B13004', 'SE_A15001', 'SE_A14007', 'SE_A10021', 'SE_A10027', 'SE_A10035', 'SE_A14002', 'SE_A14007', 'SE_A14024', 'SE_A18009', 'SE_B18002', 'SE_B12001', 'SE_A12003', 'SE_A12004', 'SE_A17005', 'SE_A17009', 'SE_A20001', 'SE_A00002']
len(table_names)

26

In [10]:
matching_columns = [col for col in df.columns.get_level_values(1) if col.startswith(tuple(table_names))]
df = df.loc[:, df.columns.get_level_values(1).isin(matching_columns)]

In [11]:
# Check for mimssing values
# df.columns[df.isna().sum() > 0 | (df.sum() > 0)]

## Processing dataset column wise

## Processing Y variable
The number of families for each zip code are available for poverty ratios less than 1, between 1 and 2 and more than 2.
All the families with ratio under 2 are considered to be poor or struggling and over 2 are considered to be ok.  
  
Based on this information, we convert this information into one column which has values 0 or 1  
0 - if more than 50% of families in the zip code have a ratio over 2  
1 - if 50% or more of families in the zip code are having ratio under 2  

In [12]:
df.loc[:, df.columns.get_level_values(1).str.startswith('SE_B13004')]

,Population for Whom Poverty Status Is Determined:,Population for Whom Poverty Status Is Determined: Under 1.00 (Doing Poorly),Population for Whom Poverty Status Is Determined: 1.00 to 1.99 (Struggling),Population for Whom Poverty Status Is Determined: Under 2.00 (Poor or Struggling),Population for Whom Poverty Status Is Determined: 2.00 and Over (Doing Ok)
,SE_B13004_001,SE_B13004_002,SE_B13004_003,SE_B13004_004,SE_B13004_005
zip_code,,,,,
10001,22983,4211,2023,6234,16749
10002,77234,21214,19285,40499,36735
10003,45455,3973,3450,7423,38032
10004,2990,115,107,222,2768
10005,8680,912,295,1207,7473
...,...,...,...,...,...
19971,13056,1002,1611,2613,10443
19973,24622,3716,4959,8675,15947


In [13]:
df['underPoverty'] = np.where(df[('Population for Whom Poverty Status Is Determined: 2.00 and Over (Doing Ok)', 'SE_B13004_005')].div(df[('Population for Whom Poverty Status Is Determined:', 'SE_B13004_001')]) < 0.5,1,0) # F13004 table contains the number of families within the different ratio ranges
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_B13004')] # Drop the columns from the table after processing

In [14]:
len(df.loc[df['underPoverty'] == 1]) # There are 317 zip codes in region 1 which have more than 50% of families that are poor or struggling

317

In [15]:
fig = go.Figure(data=go.Scattergeo(
        lon = pgeocode.Nominatim('us').query_postal_code(df.index.tolist()).longitude,
        lat = pgeocode.Nominatim('us').query_postal_code(df.index.tolist()).latitude,
        marker_color = df['underPoverty'],
        ))
fig.update_layout(
        title = 'Poverty levels of zip codes',
        geo_scope='usa',
    )
fig.show()

## Race, Population

In [16]:
df.head(2)

,Total Population,Population Density (Per Sq. Mile),Area (Land),Total Population:,Total Population: White Alone,Total Population: Black or African American Alone,Total Population: American Indian and Alaska Native Alone,Total Population: Asian Alone,Total Population: Native Hawaiian and Other Pacific Islander Alone,Total Population: Some Other Race Alone,...,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months Below Poverty Level: Male: in Labor Force,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months Below Poverty Level: Male: in Labor Force: Employed,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months Below Poverty Level: Male: in Labor Force: Unemployed,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months Below Poverty Level: Male: Not in Labor Force,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months At or Above Poverty Level,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months At or Above Poverty Level: Male: in Labor Force,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months At or Above Poverty Level: Male: in Labor Force: Employed,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months At or Above Poverty Level: Male: in Labor Force: Unemployed,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months At or Above Poverty Level: Male: Not in Labor Force,underPoverty
,SE_A00002_001,SE_A00002_002,SE_A00002_003,SE_A03001_001,SE_A03001_002,SE_A03001_003,SE_A03001_004,SE_A03001_005,SE_A03001_006,SE_A03001_007,...,SE_A13007_003,SE_A13007_004,SE_A13007_005,SE_A13007_006,SE_A13007_007,SE_A13007_008,SE_A13007_009,SE_A13007_010,SE_A13007_011,
zip_code,,,,,,,,,,,,,,,,,,,,,
10001,23947,38537.59,0.621393,23947,14923,2626,59,5242,5,565,...,1076,838,238,2064,17461,14095,13413,682,3366,0
10002,77925,88610.21,0.879413,77925,24502,5817,516,33675,5,10713,...,5082,3778,1304,12890,50144,32030,30253,1777,18114,1


In [17]:
df.drop(['SE_A00002_001','SE_A00002_003', ], axis = 1, level = 1, inplace = True)

### White Majority
We check if white-only population is the majority in a zip code  
Primarily non-white areas might be experiencing higher levels of poverty.  
0 - if less than 50% of population in the zip code are white  
1 - if more than 50% of families in the zip code are non-white


#### There are some zip codes with total population of 0. We will drop those zip codes

In [18]:
df.drop(df.loc[df.loc[:,df.columns.get_level_values(1).str.startswith('SE_A03001_001')].isin([0]).any(axis = 1)].index, inplace = True)
df.loc[:, 'whiteMajority'] = np.where((df.loc[:,'Total Population: White Alone'].values) / (df.loc[:,'Total Population:'].values) > 0.5,1,0) # If the proportion of white alone population is more than 50% we set this binary variable to 1
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A03001')] # Drop the columns from the table after processing
df['whiteMajority'].value_counts()

1    3400
0     203
Name: whiteMajority, dtype: int64

## Education

### Highest degree attainment

In [19]:
df.loc[: ,df.columns.get_level_values(1).str.startswith('SE_B12001_')]

,Population 25 Years and Over,Population 25 Years and Over: Less than High School,Population 25 Years and Over: High School Diploma,Population 25 Years and Over: Bachelor's Degree or Better
,SE_B12001_001,SE_B12001_002,SE_B12001_003,SE_B12001_004
zip_code,,,,
10001,17839,1094,4213,12532
10002,60938,20252,20497,20189
10003,39339,1377,6343,31619
10004,2409,34,195,2180
10005,6403,45,693,5665
...,...,...,...,...
19971,11208,427,5199,5582
19973,17589,3169,11531,2889


In [20]:
df = df.loc[df['Population 25 Years and Over','SE_B12001_001'] != 0] # Some of the zip codes have 0. To avoid RuntimeWarning we drop these rows

In [21]:
df.loc[:, 'eduHsOrLessProportion'] = ((df.loc[:,('Population 25 Years and Over: Less than High School','SE_B12001_002')].values) / (df.loc[:,('Population 25 Years and Over','SE_B12001_001')].values)) # Proportion of 25yo and over with less than high school education
df.loc[:, 'eduHsProportion'] = ((df.loc[:,('Population 25 Years and Over: High School Diploma','SE_B12001_003')].values) / (df.loc[:,('Population 25 Years and Over','SE_B12001_001')].values)) # Proportion of 25yo and over with less than high school education
df.loc[:, 'eduBachOrBetterProportion'] = ((df.loc[:,('Population 25 Years and Over: Bachelor\'s Degree or Better','SE_B12001_004')].values) / (df.loc[:,('Population 25 Years and Over','SE_B12001_001')].values)) # Proportion of 25yo and over with less than high school education
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_B12001_')] # Drop the columns from the table after processing
print(df.eduHsOrLessProportion.describe())
print(df.eduHsProportion.describe())
print(df.eduBachOrBetterProportion.describe())

count    3600.000000
mean        0.109179
std         0.083302
min         0.000000
25%         0.057669
50%         0.094320
75%         0.139031
max         1.000000
Name: eduHsOrLessProportion, dtype: float64
count    3600.000000
mean        0.630402
std         0.155895
min         0.000000
25%         0.555705
50%         0.664817
75%         0.730466
max         1.000000
Name: eduHsProportion, dtype: float64
count    3600.000000
mean        0.260419
std         0.174205
min         0.000000
25%         0.142920
50%         0.215010
75%         0.336710
max         1.000000
Name: eduBachOrBetterProportion, dtype: float64


### School Enrollment

In [22]:
df.loc[: ,df.columns.get_level_values(1).str.startswith('SE_A12004')] # Drop the columns from the table after processing

,Population 3 Years and Over:,Population 3 Years and Over: Enrolled in School,Population 3 Years and Over: Not Enrolled in School
,SE_A12004_001,SE_A12004_002,SE_A12004_003
zip_code,,,
10001,23517,5191,18326
10002,76232,13710,62522
10003,53777,14649,39128
10004,2934,433,2501
10005,8430,1578,6852
...,...,...,...
19971,12982,1478,11504
19973,24171,5517,18654


In [23]:
df.loc[:, 'eduEnrollmentMajority'] = np.where((df.loc[:,('Population 3 Years and Over: Enrolled in School','SE_A12004_002')].values) / (df.loc[:,('Population 3 Years and Over:','SE_A12004_001')].values) > 0.5,1,0) # If the proportion of 3yo and over enrolled in school is more than 50% we set this binary variable to 1
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A12004')] # Drop the columns from the table after processing
df['eduEnrollmentMajority'].value_counts()
df

,Population Density (Per Sq. Mile),Occupied Housing Units,Occupied Housing Units: Less than High School Graduate,Occupied Housing Units: High School Graduate (Including Equivalency),Occupied Housing Units: Some College or Associate's Degree,Occupied Housing Units: Bachelor's Degree or Higher,Civilian Population 16 to 19 Years:,"Civilian Population 16 to 19 Years: Not High School Graduate, Not Enrolled (Dropped Out)","Civilian Population 16 to 19 Years: High School Graduate, or Enrolled (In School)",Male Civilian Population 16 to 19 Years:,...,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months At or Above Poverty Level: Male: in Labor Force,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months At or Above Poverty Level: Male: in Labor Force: Employed,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months At or Above Poverty Level: Male: in Labor Force: Unemployed,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months At or Above Poverty Level: Male: Not in Labor Force,underPoverty,whiteMajority,eduHsOrLessProportion,eduHsProportion,eduBachOrBetterProportion,eduEnrollmentMajority
,SE_A00002_002,SE_A10021_001,SE_A10021_002,SE_A10021_003,SE_A10021_004,SE_A10021_005,SE_A12003_001,SE_A12003_002,SE_A12003_003,SE_A12003A_001,...,SE_A13007_008,SE_A13007_009,SE_A13007_010,SE_A13007_011,,,,,,
zip_code,,,,,,,,,,,,,,,,,,,,,
10001,38537.5900,12347,781,1107,1495,8964,1202,16,1186,366,...,14095,13413,682,3366,0,1,0.061326,0.236168,0.702506,0
10002,88610.2100,33461,9946,6638,4751,12126,2723,117,2606,1314,...,32030,30253,1777,18114,1,0,0.332338,0.336358,0.331304,0
10003,95471.9400,26699,817,908,3150,21824,7309,33,7276,2825,...,31043,30204,839,7265,0,1,0.035003,0.161239,0.803757,0
10004,5341.6770,1587,25,33,62,1467,33,0,33,19,...,2171,2097,74,357,0,1,0.014114,0.080946,0.904940,0
10005,118063.1000,4239,26,168,334,3711,144,0,144,35,...,6378,6194,184,491,0,1,0.007028,0.108231,0.884742,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19971,791.2240,6569,153,1350,1565,3501,417,32,385,159,...,6139,5869,270,5053,0,1,0.038098,0.463865,0.498037,0
19973,317.5146,9282,1573,3096,3013,1600,903,81,822,567,...,11326,10855,471,5875,0,1,0.180169,0.655580,0.164250,0


### School Dropout rate

In [24]:
df = df.loc[df['Civilian Population 16 to 19 Years:','SE_A12003_001'] != 0] # Some of the zip codes have 0 civilian population. To avoid RuntimeWarning we drop these rows

In [25]:
df.loc[:, 'eduDropoutMajority'] = np.where((df.loc[:,('Civilian Population 16 to 19 Years: Not High School Graduate, Not Enrolled (Dropped Out)','SE_A12003_002')].values) / (df.loc[:,('Civilian Population 16 to 19 Years:','SE_A12003_001')].values) > 0.5,1,0) # If the proportion of late teens dropped out from school is more than 50% we set this binary variable to 1
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A12003')] # Drop the columns from the table after processing
df.eduDropoutMajority.value_counts()

C:\Users\mohit\AppData\Local\Temp\ipykernel_14616\1463174127.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0    3237
1      42
Name: eduDropoutMajority, dtype: int64

## Employment

### Proportion of labor force employed

In [26]:
df.loc[: ,df.columns.get_level_values(1).str.startswith('SE_A17005')]

,Civilian Population in Labor Force 16 Years and Over:,Civilian Population in Labor Force 16 Years and Over: Employed,Civilian Population in Labor Force 16 Years and Over: Unemployed,Civilian Male in Labor Force 16 Years and Over:,Civilian Male in Labor Force 16 Years and Over: Employed,Civilian Male in Labor Force 16 Years and Over: Unemployed,Civilian Female in Labor Force 16 Years and Over:,Civilian Female in Labor Force 16 Years and Over: Employed,Civilian Female in Labor Force 16 Years and Over: Unemployed
,SE_A17005_001,SE_A17005_002,SE_A17005_003,SE_A17005A_001,SE_A17005A_002,SE_A17005A_003,SE_A17005B_001,SE_A17005B_002,SE_A17005B_003
zip_code,,,,,,,,,
10001,15255,14311,944,9082,8624,458,6173,5687,486
10002,37185,34104,3081,20280,18254,2026,16905,15850,1055
10003,34690,32998,1692,18100,17320,780,16590,15678,912
10004,2181,2101,80,980,938,42,1201,1163,38
10005,6981,6795,186,3192,3187,5,3789,3608,181
...,...,...,...,...,...,...,...,...,...
19971,6470,6178,292,3379,3160,219,3091,3018,73
19973,12279,11574,705,6403,6006,397,5876,5568,308


In [27]:
df = df.loc[df.loc[:,('Civilian Population in Labor Force 16 Years and Over:','SE_A17005_001')] != 0] # Some of the zip codes have 0 civilian population. To avoid RuntimeWarning we drop these rows

In [28]:
# df.loc[:, 'employedMajority'] = np.where((df.loc[:,('Civilian Population in Labor Force 16 Years and Over: Employed' ,'SE_A17005_002')].values) / (df.loc[:,('Civilian Population in Labor Force 16 Years and Over:','SE_A17005_001')].values) > 0.5,1,0) # If the proportion of late teens dropped out from school is more than 50% we set this binary variable to 1
df.loc[:, 'employedProportion'] = ((df.loc[:,('Civilian Population in Labor Force 16 Years and Over: Employed' ,'SE_A17005_002')].values) / (df.loc[:,('Civilian Population in Labor Force 16 Years and Over:','SE_A17005_001')].values))
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A17005')] # Drop the columns from the table after processing
df.employedProportion.describe()

count    3277.000000
mean        0.936583
std         0.041227
min         0.595556
25%         0.921299
50%         0.943723
75%         0.961305
max         1.000000
Name: employedProportion, dtype: float64

### Employment by sector IN PROGRESS

In [29]:
df.loc[: ,df.columns.get_level_values(1).str.startswith('SE_A17009')]

,Employed Civilian Population 16 Years and Over:,Employed Civilian Population 16 Years and Over: Private Sector,Employed Civilian Population 16 Years and Over: Public Sector,Employed Civilian Population 16 Years and Over: Self-Employed (Incorporated and Not Incorporated),Employed Civilian Population 16 Years and Over: Private Non-Profit,Employed Civilian Population 16 Years and Over: Unpaid Family Workers
,SE_A17009_001,SE_A17009_002,SE_A17009_003,SE_A17009_004,SE_A17009_005,SE_A17009_006
zip_code,,,,,,
10001,14311,10003,871,1927,1491,19
10002,34104,24046,3207,3815,2807,229
10003,32998,23351,2017,4315,3255,60
10004,2101,1537,123,266,175,0
10005,6795,5651,222,474,448,0
...,...,...,...,...,...,...
19971,6178,3895,848,990,445,0
19973,11574,8299,1315,1067,871,22


In [30]:
df.loc[:, 'employedPvtSecProportion'] = ((df.loc[:,('Employed Civilian Population 16 Years and Over: Private Sector' ,'SE_A17009_002')].values) / (df.loc[:,('Employed Civilian Population 16 Years and Over:','SE_A17009_001')].values))
df.loc[:, 'employedPubSecProportion'] = ((df.loc[:,('Employed Civilian Population 16 Years and Over: Public Sector' ,'SE_A17009_003')].values) / (df.loc[:,('Employed Civilian Population 16 Years and Over:','SE_A17009_001')].values))
df.loc[:, 'employedSelfEmpProportion'] = ((df.loc[:,('Employed Civilian Population 16 Years and Over: Self-Employed (Incorporated and Not Incorporated)' ,'SE_A17009_004')].values) / (df.loc[:,('Employed Civilian Population 16 Years and Over:','SE_A17009_001')].values))
df.loc[:, 'employedPvtNonProProportion'] = ((df.loc[:,('Employed Civilian Population 16 Years and Over: Private Non-Profit' ,'SE_A17009_005')].values) / (df.loc[:,('Employed Civilian Population 16 Years and Over:','SE_A17009_001')].values))
df.loc[:, 'employedUnpaidFamProportion'] = ((df.loc[:,('Employed Civilian Population 16 Years and Over: Unpaid Family Workers' ,'SE_A17009_006')].values) / (df.loc[:,('Employed Civilian Population 16 Years and Over:','SE_A17009_001')].values))
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A17009')] # Drop the columns from the table after processing


## Health

In [31]:
df.loc[: ,df.columns.get_level_values(1).str.startswith('SE_A20001')]

,Total:,Total: No Health Insurance Coverage,Total: with Health Insurance Coverage,Total: with Health Insurance Coverage: Public Health Coverage,Total: with Health Insurance Coverage: Private Health Insurance
,SE_A20001_001,SE_A20001_002,SE_A20001_003,SE_A20001_004,SE_A20001_005
zip_code,,,,,
10001,23947,1375,22572,6206,18258
10002,77682,5521,72161,41417,35670
10003,54902,2441,52461,9091,47385
10004,2990,195,2795,225,2712
10005,8676,375,8301,531,8056
...,...,...,...,...,...
19971,13084,690,12394,5859,10150
19973,24688,2046,22642,10963,15603


In [32]:
df = df.loc[df.loc[:,('Total:','SE_A20001_001')] != 0] # Some of the zip codes have 0. To avoid RuntimeWarning we drop these rows
df.loc[:, 'insuredProportion'] = ((df.loc[:,('Total: with Health Insurance Coverage' ,'SE_A20001_003')].values) / (df.loc[:,('Total:','SE_A20001_001')].values))
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A20001')] # Drop the columns from the table after processing
df.insuredProportion.describe()

count    3276.000000
mean        0.929536
std         0.056749
min         0.366816
25%         0.914425
50%         0.940770
75%         0.961448
max         1.000000
Name: insuredProportion, dtype: float64

In [33]:
df.shape

(3276, 175)

## Housing

### Proportion of renters spending more than 30% of income on rent

In [34]:
df.loc[:, df.columns.get_level_values(1).str.startswith('SE_B18002')]

,Renter-Occupied Housing Units,Renter-Occupied Housing Units: 30 to 49 Percent,Renter-Occupied Housing Units: 50 percent or More
,SE_B18002_001,SE_B18002_002,SE_B18002_003
zip_code,,,
10001,8868,2068,1797
10002,28951,7409,6838
10003,17979,3504,4126
10004,1113,246,164
10005,3461,743,538
...,...,...,...
19971,1079,100,250
19973,2615,711,537


In [35]:
df = df.loc[df.loc[:, ('Renter-Occupied Housing Units', 'SE_B18002_001')] != 0] # Remove observations with 0 to avoid error while dividing
df.loc[:, 'housingRentMoreThan30pcProportion'] = ((df.loc[:,('Renter-Occupied Housing Units: 30 to 49 Percent' ,'SE_B18002_002')].values + df.loc[:,('Renter-Occupied Housing Units: 50 percent or More' ,'SE_B18002_003')].values ) / (df.loc[:,('Renter-Occupied Housing Units','SE_B18002_001')].values))
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_B18002')] # Drop the columns from the table after processing


## Income

### Below Poverty line
Poverty Status in the Past 12 Months based on income level. Proportion of population having income below poverty level


In [36]:
df.loc[:, df.columns.get_level_values(1).str.startswith('SE_A13007')]

,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months Below Poverty Level,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months Below Poverty Level: Male: in Labor Force,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months Below Poverty Level: Male: in Labor Force: Employed,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months Below Poverty Level: Male: in Labor Force: Unemployed,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months Below Poverty Level: Male: Not in Labor Force,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months At or Above Poverty Level,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months At or Above Poverty Level: Male: in Labor Force,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months At or Above Poverty Level: Male: in Labor Force: Employed,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months At or Above Poverty Level: Male: in Labor Force: Unemployed,Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months At or Above Poverty Level: Male: Not in Labor Force
,SE_A13007_001,SE_A13007_002,SE_A13007_003,SE_A13007_004,SE_A13007_005,SE_A13007_006,SE_A13007_007,SE_A13007_008,SE_A13007_009,SE_A13007_010,SE_A13007_011
zip_code,,,,,,,,,,,
10001,20601,3140,1076,838,238,2064,17461,14095,13413,682,3366
10002,68116,17972,5082,3778,1304,12890,50144,32030,30253,1777,18114
10003,42164,3856,1446,923,523,2410,38308,31043,30204,839,7265
10004,2594,66,10,4,6,56,2528,2171,2097,74,357
10005,7781,912,603,601,2,309,6869,6378,6194,184,491
...,...,...,...,...,...,...,...,...,...,...,...
19971,12105,913,331,309,22,582,11192,6139,5869,270,5053
19973,19639,2438,953,719,234,1485,17201,11326,10855,471,5875


In [37]:
df['incomeBelowPovertyLevelProportion'] = ((df[('Civilian Population 16 Years and Over for Whom Poverty Status Is Determined: Income in the Past 12 Months Below Poverty Level', 'SE_A13007_002')].values) /(df[('Civilian Population 16 Years and Over for Whom Poverty Status Is Determined', 'SE_A13007_001')]))
df.incomeBelowPovertyLevelProportion.describe()
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A13007')] # Drop the columns from the table after processing


### Median Income

In [38]:
df.loc[:, 'incomeMedian'] = df.loc[: ,df.columns.get_level_values(1).str.startswith('SE_A14006_001')].values
df.incomeMedian.describe()
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A14006')] # Drop the columns from the table after processing


### Per Capita

In [39]:
df.loc[:, 'incomePerCapita'] = df.loc[: ,df.columns.get_level_values(1).str.startswith('SE_A14024_001')].values
df.incomePerCapita.describe()
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A14024')] # Drop the columns from the table after processing

### Gini Index

In [40]:
df.loc[:, 'incomeGiniIndex]'] = df.loc[: ,df.columns.get_level_values(1).str.startswith('SE_A14028_001')] # Drop the columns from the table after processing
df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A14028')] # Drop the columns from the table after processing

In [41]:
df.shape

(3196, 154)

In [42]:
df.to_csv('./2017df.csv')